# Encoding

Encoding is a way of representing information for machines in a way so that they are computationally understood. Whether or not encoding is nessessary is ususally dependent on the algorithm at hand. For example, Random Forest Classifiers are able to handle or naive bayes are classifiers that are able directly handle multiple classifications where as others such as Supper Vector Machines and Linear Classifiers only only strictly handle binary classifications.

As an even more direct example, a one hot encoder may be nessessary in a situation where you are attempting to draw a multiclassification precision recall curve. In a normal setting a `precision_recall_curve` operates on a binary classification to return the TP, TN, FP, FN attributes. Using a One vs all, or one vs rest approach applied with a one hot encoder than situation a binary situation for each classification.

In the process on cleaning your data, it is generally a good idea to encode it (you really cant go wrong with it) because most ml algorithms prefer working with numerical fields.

## Label Encoder

A label encoder is used to convert categorical variables into discreet integer representations. Even though some categorical variables are numeric, they are identified as categorical if the order of numerical value is ambigious. Some categorical features are considered to be ordinal like "low", "medium", and "high" - but a label encoder treat all categorical variables just as being categorical.

**Example**

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn import preprocessing

print('Label Encoder - single column')
example11 = ['CA', 'AR', 'NY', 'CA']
example11_df = pd.DataFrame(example11, columns=['state'])
print(tabulate(example11_df, headers='keys', tablefmt='pipe'), end='\n\n')

enc = preprocessing.LabelEncoder()
example11_enc = enc.fit_transform(example11)
print(example11_enc, end='\n\n')

print('Label Encoder - two column')
example12 = np.matrix([['A', 'D'],
                       ['B', 'E'],
                       ['C', 'D'],
                       ['B', 'F'],
                       ['A', 'E']])
example12_df = pd.DataFrame(example12, columns=['x1', 'x2'])
print(tabulate(example12_df, headers='keys', tablefmt='pipe'), end='\n\n')

# by default a sklearns label encoder only takes in a single column as an argument
# but it can be extended to perform as a multi-label-encoder
example12_enc = example12_df.apply(enc.fit_transform)
print(example12_enc, end='\n\n')

Label Encoder - single column
|    | state   |
|---:|:--------|
|  0 | CA      |
|  1 | AR      |
|  2 | NY      |
|  3 | CA      |

[1 0 2 1]

Label Encoder - two column
|    | x1   | x2   |
|---:|:-----|:-----|
|  0 | A    | D    |
|  1 | B    | E    |
|  2 | C    | D    |
|  3 | B    | F    |
|  4 | A    | E    |

   x1  x2
0   0   0
1   1   1
2   2   0
3   1   2
4   0   1



Label Encoder works uniquely, but arbitrarily on a per column basis. But there is an easy work around that.


In [2]:
# the process goes as: raveling the matrix into a single array, converting the values, and then 
# reshaping it back to its original shape.
X_str = np.array([['a', 'b'],
                  ['c', 'a'],
                  ['h', 'y'],
                  ['a', 'w']])
lenc = preprocessing.LabelEncoder()
X_int = lenc.fit_transform(X_str.ravel()).reshape(*X_str.shape)

We can map the numerical fields back to their string representation by indexing into `encoder.classes_`. This is an array where the index holds the numerical representation of the data, and the value holds the mapping class.

### When is Integer Based Encoding Insufficient?

With integer based encoding (such as with Label Encoding), order can be assumed because the values, although unit based, are still numeric. This implies order, when categories are orderless, have no concept of distance and are nonmeasurable by nature.


## One Hot Encoder

One hot encoders are a method of representing categorical variables into a unique binary sequence. It is called "one hot" encoder because there is a unique index activating per sequence (originally applied in digital circuits). In otherwords, the only option for for a hot encoder a binary indicator. A value is either off, or on, and it is used in scenarios where strict binary machine learning classifiers need to perform a multiclassfication.

### One vs Rest Classification

The strategy for this is simple. We build any many binary detectors as there are classes. For example, for a digit detection system, we build a individual detectors to recognize between and 0 and not a 0, a 1 and not a 1, a 2 and not a 2, and so on and so fourth. Then to perfrom a classification, you use each binary classifier and output a decision score. The chosen classification would then become binary classifier with the maximum (or minimum, depending on the interpretability) decision score.

### One vs One Classification

Although, OvsR is general preferred most machine learning applications, there is another approach called one vs one that is worth understanding. In a one vs one classification, a classifer is built for all combinational pairs of classes. For example given 4 classes, 4C2 or 6 classifiers would be built to distinguish between a (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), and (2, 3). In this context a classifier is aware of only two different things that exist. By classifing between combinational pairs of two, we in effect create a comparision between every kind of classification. For example, we can distinguish a 0 from a 1, 2, or 3. Or we can distinguish a 1 from a 0, 2, or 3. The advantage here is that a classifier trains only against the dependent classes, and so each classifier can potentially get quickly trained when the alternative to have a larger dataset wouldn't scale so well against the classifer (for example, an SVM). The disadvantage is the the number of classifiers grow $O(N^2)$ or more precisely $O(\frac{N(N-1)}{2})$ where N is the number of unique classes.

By default OHE returns a sparse matrix. A sparse matrix only stores the position of a non-zero value in a matrix. Alternatively, we can convert a sparse matrix into a regular matrix using `toarray()`.


**By default**, sklearn will automatically detect a multilabled classification and perform one vs rest, or one vs one within the context of the classifier automatically. However, we can still force the use of a particular multilabeled encoding scheme using 

```python
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
```


In [10]:
# note: transform is required in order treat as a column vector
ohe = preprocessing.OneHotEncoder(sparse=False, categories="auto")

print('OH Encoder - single column')
example21 = np.matrix(["1", "0", "2", "1"]).T
example21_enc = ohe.fit_transform(example21)
print(example21, end='\n\n')
print(example21_enc)
print('vocabulary:', ohe.categories_, end='\n\n')


print('OH Encoder - two column')
example22 = np.matrix([["0", "0"],
                       ["1", "1"],
                       ["2", "0"],
                       ["1", "2"],
                       ["0", "4"]])
example22_enc = ohe.fit_transform(example22)
print(example22, end='\n\n')
print(example22_enc, end='\n')
print('vocabulary:', ohe.categories_, end='\n\n')


OH Encoder - single column
[['1']
 ['0']
 ['2']
 ['1']]

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
vocabulary: [array(['0', '1', '2'], dtype='<U1')]

OH Encoder - two column
[['0' '0']
 ['1' '1']
 ['2' '0']
 ['1' '2']
 ['0' '4']]

[[1. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1.]]
vocabulary: [array(['0', '1', '2'], dtype='<U1'), array(['0', '1', '2', '4'], dtype='<U1')]



**Observations:**

* One hot encoder also works uniquely and arbitrarily on a per column basis, but just concatenates the results column wise to a single matrix. We can see how this can explode the feature space.

**Why is it useful?**

Many machine learning algorithms rely on applying a single weight per feature, or compute distances between samples $x_i$. We can see how this might be a problem with categorical variables. For example, suppose we have label encoded a categorical feature to be one of [0, 1, 2]. 

The first issue with this is that the distances are not equivalent, as they often should be. The language "english" for example as a variable should be indifferent to "spanish" or "japanese". With the one-hot encoding however, the pairwise distances between [1, 0, 0], [0, 1, 0] and [0, 0, 1] are all equal to $\sqrt{2}$. So in affect, the issue of order or distance is aleivated while preserving an unique categorical mapping or reinterpretation.

The next issue is similar but seen in a different light. Given a weight for a feature $x$, decisions are often made based off the constraint on $w*x + b < 0$ or $w*x + b \geq 0$. One problem with this is that in the context of $x \in [0,1,2]$, $w*1 + b$ and $w*2 + b$ will both lead to the same conclusion despite being potentially very different things. In other words, this model does not encode a three-way choice. So now, we have the option to changing the model, or changing the variable to make things work out. 

One-hot-encoding blows up the feature space proportional to the number of unique instances in a single column. Because all the variables are now boolean, we obtain the possibility of both suppressing and activating certain variables to model a three way decision.

By one-hot encoding, you effectively blow up the feature space to three features, which will each get their own weights, so the decision function is now w[UK]x[UK] + w[FR]x[FR] + w[US]x[US] < b, where all the x's are booleans. In this space, such a linear function can express any sum/disjunction of the possibilities (e.g. "UK or US", which might be a predictor for someone speaking English).
